# Evaluating each RAG pipeline using giskard

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file using 'from dotenv import find_dotenv, load_dotenv'
load_dotenv(find_dotenv(filename='SURF-Project_Optimizing-PerunaBot/setup/.env'))

openai_api_key = os.environ['OPENAI_API_KEY']

In [4]:
print(os.environ["LANGCHAIN_TRACING_V2"])

false


In [63]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-949fa2fd-4b60-4a2f-a28a-22b309dcbb4f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
from OG_PerunaBot_chain import Original_PerunaBot_eval_chain
from chain_0 import base_retriever_eval_chain_0
from chain_1 import parent_retriever_eval_chain_1
from chain_2 import ensemble_retriever_eval_chain_2

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# file paths of PDFs to be used
pdf_paths = ['../Data/Evaluation Data/Southern Methodist University - 2023-2024 Undergraduate Catalog from About SMU to Right to Know.pdf',
             '../Data/Evaluation Data/Important University Resources from SMU Student Handbook 23-24.pdf',
             '../Data/Evaluation Data/Important SMU Numbers and Websites.pdf'
             ]

# Function to load PDFs using LangChain's PyPDFLoader
def load_pdfs_with_langchain(pdf_paths):
    documents = []
    for path in pdf_paths:
        try:
            # Use LangChain's PyPDFLoader to load the PDF
            loader = PyPDFLoader(path)
            # Load and pase the PDF into document instances
            pdf_doc = loader.load()
            # Insert the parsed PDF documents into the documents list
            documents.extend(pdf_doc)
        except Exception as e:
            print(f"Error loading {path}: {e}")
    return documents

# Load PDF documents using the function
evaluation_pdf_docs = load_pdfs_with_langchain(pdf_paths)

print(len(evaluation_pdf_docs))
print(evaluation_pdf_docs[0].page_content[0:100])
print(evaluation_pdf_docs[7].metadata)

93
14 
 About SMU  
The Vision of Southern Methodist University  
To create and impart knowledge that w
{'source': '../Data/Evaluation Data/Southern Methodist University - 2023-2024 Undergraduate Catalog from About SMU to Right to Know.pdf', 'page': 7}


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, 
                                                length_function=len, add_start_index=True)  

# Split the PDF documents into chunks using the text splitter
split_evaluation_pdf_docs = text_splitter.split_documents(evaluation_pdf_docs)
print(len(split_evaluation_pdf_docs))

395


In [9]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in split_evaluation_pdf_docs], columns = ['text'])
df.head(10)

,text
0,14 \n About SMU \nThe Vision of Southern Meth...
1,"Metroplex, Southern Methodist University seeks..."
2,Education and Human Development. \nFounded in...
3,"The full -time equivalent enrollment was 8,703..."
4,"university, but a great university."
5,15 \n Academic Accreditation \nSouthern Method...
6,Dedman School of Law \n(7-year cycle) The Am...
7,Dance \n(10-year cycle) National Association...
8,16 \n UNIT State/Regional/National Accreditin...
9,17 \n Admission \nSouthern Methodist Universi...


In [10]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

In [22]:
prompt = """
    You are PerunaBot, an AI assistant trained on domain-specific information on
    Southern Methodist University.  Your primary role is to provide detailed, accurate, 
    and helpful responses to questions based on the following retrieved context. 
    You assist faculty, administrators, prospective students, and current students by 
    offering precise and specific answers. Maintain a welcoming and friendly tone. 
    Try to keep responses to 3-5 sentences maximum unless more detail is requested.
"""

from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base=knowledge_base,
    num_questions=25,
    agent_description=prompt,
)

2024-08-01 16:51:11,255 pid:11724 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-08-01 16:52:31,478 pid:11724 MainThread giskard.rag  INFO     Found 19 topics in the knowledge base.


Generating questions:   0%|          | 0/25 [00:00<?, ?it/s]

In [23]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(5).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference Answer: {row[1]['reference_answer']}")
    print("Reference Context:")
    print(row[1]['reference_context'])
    print("********************", end="\n\n")



Question 1: Where can I find information about the certification process?
Reference Answer: Information about the certification process is available in the Enrollment and Academic Records section of the catalog, as well as the www.smu.edu/EnrollmentServices/Veterans website.
Reference Context:
Document 95: certification process is available in the Enrollment and Academic Records  section of this catalog, as well as the 
www.smu.edu/EnrollmentS ervices/Veterans website.
********************

Question 2: What is the policy for students who have been reinstated to the University following suspension?
Reference Answer: Students who have been reinstated to the University following suspension remain on probation and are normally allowed two regular terms within which they can make up their academic deficiencies and return to good standing. However, special conditions for the first term may be set in individual cases. If they fail to meet the terms of academic probation, they will be suspende

In [24]:
testset.save("../Data/Evaluation Data/giskard-testset_1.jsonl")

In [37]:
def predict_OG_chain(question, history=None):
    response = Original_PerunaBot_eval_chain.invoke({"question": question})
    return response["output"]

def predict_chain_0(question, history=None):
    response = base_retriever_eval_chain_0.invoke({"question": question})
    return response["output"]

def predict_chain_1(question, history=None):
    response = parent_retriever_eval_chain_1.invoke({"question": question})
    return response["output"]

def predict_chain_2(question, history=None):
    response = ensemble_retriever_eval_chain_2.invoke({"question": question})
    return response["output"]

In [28]:
from giskard.rag import evaluate

In [38]:
OG_PerunaBot_report = evaluate(
    answer_fn=predict_OG_chain,
    testset=testset,
    knowledge_base=knowledge_base
)
display(OG_PerunaBot_report)

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...

In [39]:
OG_PerunaBot_report.to_html("../Data/Evaluation Results/OG_PerunaBot_giskard_report_8-1.html")

In [55]:
OG_PerunaBot_report.correctness_by_question_type()

,correctness
question_type,
complex,0.75
conversational,0.25
distracting element,0.25
double,0.75
simple,1.00
situational,0.75


In [56]:
OG_PerunaBot_report.correctness_by_topic()

,correctness
topic,
Academic Credit and Placement,1.000000
Others,0.333333
Southern Methodist University Life,0.750000
University Academic Policies,0.500000
University Admission Process,1.000000
University Admission and Credit Transfer,1.000000
University Admissions and English Proficiency,0.000000
University Credit and Course Policies,0.000000
University Education Curriculum,0.000000


In [40]:
PerunaBot_0_report = evaluate(
    answer_fn=predict_chain_0,
    testset=testset,
    knowledge_base=knowledge_base
)
display(PerunaBot_0_report)

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...

In [41]:
PerunaBot_0_report.to_html("../Data/Evaluation Results/PerunaBot_0_giskard_report_8-1.html")

In [53]:
PerunaBot_0_report.correctness_by_question_type()

,correctness
question_type,
complex,1.00
conversational,0.50
distracting element,0.50
double,1.00
simple,0.80
situational,0.75


In [54]:

PerunaBot_0_report.correctness_by_topic()

,correctness
topic,
Academic Credit and Placement,1.000000
Others,0.666667
Southern Methodist University Life,1.000000
University Academic Policies,0.000000
University Admission Process,1.000000
University Admission and Credit Transfer,1.000000
University Admissions and English Proficiency,0.500000
University Credit and Course Policies,1.000000
University Education Curriculum,0.000000


In [42]:
PerunaBot_1_report = evaluate(
    answer_fn=predict_chain_1,
    testset=testset,
    knowledge_base=knowledge_base
)
display(PerunaBot_1_report)

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...

In [43]:
PerunaBot_1_report.to_html("../Data/Evaluation Results/PerunaBot_1_giskard_report_8-1.html")

In [51]:
PerunaBot_1_report.correctness_by_question_type()

,correctness
question_type,
complex,1.00
conversational,0.25
distracting element,0.75
double,0.75
simple,1.00
situational,1.00


In [52]:

PerunaBot_1_report.correctness_by_topic()

,correctness
topic,
Academic Credit and Placement,1.000000
Others,0.666667
Southern Methodist University Life,0.500000
University Academic Policies,0.500000
University Admission Process,1.000000
University Admission and Credit Transfer,1.000000
University Admissions and English Proficiency,1.000000
University Credit and Course Policies,1.000000
University Education Curriculum,1.000000


In [45]:
PerunaBot_2_report = evaluate(
    answer_fn=predict_chain_2,
    testset=testset,
    knowledge_base=knowledge_base
)

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

In [46]:
display(PerunaBot_2_report)

Loading BokehJS ...

In [47]:
PerunaBot_2_report.to_html("../Data/Evaluation Results/PerunaBot_2_giskard_report_8-1.html")

In [49]:
PerunaBot_2_report.correctness_by_question_type()


,correctness
question_type,
complex,1.00
conversational,0.00
distracting element,0.75
double,1.00
simple,0.80
situational,1.00


In [50]:

PerunaBot_2_report.correctness_by_topic()

,correctness
topic,
Academic Credit and Placement,1.000000
Others,0.666667
Southern Methodist University Life,0.500000
University Academic Policies,0.500000
University Admission Process,1.000000
University Admission and Credit Transfer,1.000000
University Admissions and English Proficiency,1.000000
University Credit and Course Policies,1.000000
University Education Curriculum,1.000000


In [11]:
from OG_PerunaBot_chain import Original_PerunaBot_eval_chain_v1
from chain_0 import base_retriever_eval_chain_0_v1
from chain_1 import parent_retriever_eval_chain_1_v1
from chain_2 import ensemble_retriever_eval_chain_2_v1

In [12]:
prompt_v1 = """
    You are Dr. Alex, a university academic advisor with over 15 years
    of experience in higher education as a academic advisor at Southern
    Methodist University. You have engaged with hundreds of students and 
    understand the ins and outs of the university's academic programs,
    policies, and procedures. You are going to mimic the role of 
    prospective and current students, asking questions about the university.
    Then you will provide detailed, accurate, and specific answers 
    to these questions based on your knowledge. Try to keep answers to 
    3-5 sentences unless more detail is requested.
"""

from giskard.rag import generate_testset

testset_2 = generate_testset(
    knowledge_base=knowledge_base,
    num_questions=25,
    agent_description=prompt_v1
)

2024-08-05 08:06:16,941 pid:432 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-08-05 08:08:03,255 pid:432 MainThread giskard.rag  INFO     Found 23 topics in the knowledge base.


Generating questions:   0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
test_set_df_2 = testset_2.to_pandas()   

for index, row in enumerate(test_set_df_2.head(5).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference Answer: {row[1]['reference_answer']}")
    print("Reference Context:")
    print(row[1]['reference_context'])
    print("********************", end="\n\n")

Question 1: What is the focus of the Fort Burgwin Library?
Reference Answer: The Fort Burgwin Library is focused on the history, literature, cultures and environment of New Mexico and the Southwest.
Reference Context:
Document 344: data from premier providers. The Business Library includes the Kitt Investing and Trading Center, quiet and group 
study areas, a periodicals area, facility wide wireless access, more than 400 electronic resources, th e Hillcrest 
Foundation International Resource Library, the Edwin L. Cox Business Leadership Center Resource Collection, the 
Maguire Energy Institute Resource Collection and the Cox Career Services Collection. Librarians are available all 
hours that the bus iness library is open, providing research assistance both in person and virtually.  
Fort Burgwin Library 
The Fort Burgwin Library, located in Taos, New Mexico, serves students and faculty in the SMU -in-Taos program. 
It is focused on the history, literature, cultures and environment of 

In [14]:
testset_2.save("../Data/Evaluation Data/giskard-testset_2.jsonl")

In [15]:
chain_experiment_v1 = {
    "Original PerunaBot v1": Original_PerunaBot_eval_chain_v1,
    "PerunaBot 0 v1": base_retriever_eval_chain_0_v1,
    "PerunaBot 1 v1": parent_retriever_eval_chain_1_v1,
    "PerunaBot 2 v1": ensemble_retriever_eval_chain_2_v1
}


In [18]:
evaluation_prompt = """
    You are Dr. Alexa Morgana, a university academic advisor with over 15 years
    of experience in higher education as an academic advisor at Southern Methodist
    University. You have engaged with hundreds of students and understand the ins
    and outs of the university's academic programs, policies, and procedures. You
    are in the process of training academic advisors at the university to provide
    detailed, accurate, and specific answers to prospective and current students'
    questions. Evaluate the responses to the following questions based on these
    criteria: helpfulness, relevance, accuracy, depth, and level of detail.
"""

In [19]:
from giskard.rag import evaluate
from giskard.rag import metrics

In [21]:
for name in chain_experiment_v1:
    def predict_chain(question, history=None):
        chain = chain_experiment_v1[name]
        response = chain.invoke({"question": question})
        return response["output"]
    
    results = evaluate(
        answer_fn=predict_chain,
        testset=testset_2,
        knowledge_base=knowledge_base,
        agent_description = evaluation_prompt
    )
    
    display(results)
    results.to_html(f"../Data/Evaluation Results/{name}_giskard_report_8-1.html")
    results.correctness_by_question_type()
    results.correctness_by_topic()
    results.get_failures()

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

...

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...

Asking questions to the agent:   0%|          | 0/25 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Loading BokehJS ...